In [3]:
from pathlib import Path

result_dir = Path("runs/llama31")
result_dir.mkdir(parents=True, exist_ok=True)   # <- create if missing


In [4]:
!mkdir -p runs/genre_expand_neurons_full
!mkdir -p runs/movie_genre_neurons_full
!ls runs/


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


genre_expand_neurons	   movie_genre_neurons	     neurons_by_genre.json
genre_expand_neurons_full  movie_genre_neurons_full


In [2]:
!grep -n "autograd.grad" keyneuron/neuron_attribution/_neuron_attribution.py
!grep -n ".backward" keyneuron/neuron_attribution/_neuron_attribution.py


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


438:                grad = torch.autograd.grad(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
import os, json
from keyneuron import KeyNeuron

os.chdir("/data/susheel/qrneuron-master/qrneuron-master")

# 1) Movie → Genre
with open("data/movie_genre_mcq_full.json", "r", encoding="utf-8") as f:
    movie_mcq = json.load(f)

kn = KeyNeuron(
    model_name="meta-llama/Meta-Llama-3.1-8B-Instruct",
    data_samples=movie_mcq,
    result_dir="runs/movie_genre_neurons_full/",
    common_threshold=0.3,
    top_v=20,
    option_letters=["A","B","C","D"],
)
kn._extract_key_neuron()

# 2) Genre Expand
with open("data/genre_expand_mcq_full.json", "r", encoding="utf-8") as f:
    expand_mcq = json.load(f)

kn = KeyNeuron(
    model_name="meta-llama/Meta-Llama-3.1-8B-Instruct",
    data_samples=expand_mcq,
    result_dir="runs/genre_expand_neurons_full/",
    common_threshold=0.3,
    top_v=20,
    option_letters=["A","B","C","D"],
)
kn._extract_key_neuron()




Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2025-11-05 10:20:40,181 - keyneuron.neuron_attribution._neuron_attribution - INFO - LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
     

22


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2025-11-05 16:11:57,430 - keyneuron.neuron_attribution._neuron_attribution - INFO - LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
     

20


In [1]:
import os, json
from keyneuron import KeyNeuron

os.chdir("/data/susheel/qrneuron-master/qrneuron-master")

with open("data/movie_genre_mcq.json", "r", encoding="utf-8") as f:
    data_samples = json.load(f)  # dict: id -> {question, options, answer}

print(type(data_samples), len(data_samples))  # should be dict, 10

kn = KeyNeuron(
    model_name="meta-llama/Meta-Llama-3.1-8B-Instruct",
    data_samples=data_samples,                 # ✅ dict, not list
    result_dir="runs/llama31/",
    common_threshold=0.3,
    top_v=20,
    option_letters=["A","B","C","D"],
)

kn._extract_key_neuron()

<class 'dict'> 10


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2025-11-04 23:41:33,808 - keyneuron.neuron_attribution._neuron_attribution - INFO - LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
     

10


In [2]:
import os, json
from keyneuron import KeyNeuron

os.chdir("/data/susheel/qrneuron-master/qrneuron-master")

with open("data/genre_expand_mcq.json", "r", encoding="utf-8") as f:
    data_samples = json.load(f)  # dict: id -> {question, options, answer}

print(type(data_samples), len(data_samples))  # should be dict, 10

kn = KeyNeuron(
    model_name="meta-llama/Meta-Llama-3.1-8B-Instruct",
    data_samples=data_samples,                 # ✅ dict, not list
    result_dir="runs/genre_expand_neurons/",
    common_threshold=0.3,
    top_v=20,
    option_letters=["A","B","C","D"],
)

kn._extract_key_neuron()

<class 'dict'> 10


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2025-11-05 01:07:12,385 - keyneuron.neuron_attribution._neuron_attribution - INFO - LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
     

10


In [6]:
import json
from collections import defaultdict
from pathlib import Path

# 1. Paths – change if your folders are named slightly differently
movie_mcq_path   = Path("data/movie_genre_mcq_full.json")
expand_mcq_path  = Path("data/genre_expand_mcq_full.json")

movie_keys_path  = Path("runs/movie_genre_neurons_full/key_neuron.json")
expand_keys_path = Path("runs/genre_expand_neurons_full/key_neuron.json")

# 2. Load MCQ files (dict: uuid -> sample)
with movie_mcq_path.open("r", encoding="utf-8") as f:
    movie_mcq = json.load(f)

with expand_mcq_path.open("r", encoding="utf-8") as f:
    expand_mcq = json.load(f)

def build_uuid_to_genre(mcq_dict):
    """
    mcq_dict: {"mgq_001": {"question":..., "options": {...}, "answer": "A"}, ...}
    returns: {"mgq_001": "Action", "mgq_002": "Comedy", ...}
    """
    mapping = {}
    for uid, sample in mcq_dict.items():
        options = sample["options"]
        correct_letter = sample["answer"]
        genre = options[correct_letter].strip()
        mapping[uid] = genre
    return mapping

movie_uuid2genre  = build_uuid_to_genre(movie_mcq)
expand_uuid2genre = build_uuid_to_genre(expand_mcq)

# 3. Collect neurons per genre from key_neuron.json
#    We'll store them as a set of (layer, neuron_idx) for each genre.
genre2neurons = defaultdict(set)

def add_keyneurons_from_file(key_path: Path, uuid2genre: dict):
    """
    key_path: path to key_neuron.json (one JSON object per line)
    uuid2genre: mapping from uuid -> genre string
    """
    with key_path.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            obj = json.loads(line)
            uid = obj["uuid"]
            neurons = obj["neurons"]   # list like [[layer, idx], [layer, idx], ...]

            genre = uuid2genre.get(uid)
            if genre is None:
                # this uuid not in mcq dict (shouldn't happen, but just in case)
                continue

            for layer, idx in neurons:
                genre2neurons[genre].add((layer, idx))

# add from both runs
add_keyneurons_from_file(movie_keys_path,  movie_uuid2genre)
add_keyneurons_from_file(expand_keys_path, expand_uuid2genre)

# 4. Convert sets to sorted lists for JSON saving
neurons_by_genre = {}
for genre, neuron_set in genre2neurons.items():
    # convert (layer, idx) -> [layer, idx]
    neurons_by_genre[genre] = sorted([[l, i] for (l, i) in neuron_set])

out_path = Path("runs/neurons_by_genre.json")
with out_path.open("w", encoding="utf-8") as f:
    json.dump(neurons_by_genre, f, indent=2)

print("✅ Saved neurons_by_genre to", out_path)
for g, ns in list(neurons_by_genre.items())[:5]:
    print(g, "→", len(ns), "neurons")


✅ Saved neurons_by_genre to runs/neurons_by_genre.json
Children's → 20 neurons
Comedy → 40 neurons
Documentary → 56 neurons
Drama → 60 neurons
Fantasy → 51 neurons


In [18]:
import json, os

os.chdir("/data/susheel/qrneuron-master/qrneuron-master")

with open("data/domain_sample_multi_choice_qa.json", "r", encoding="utf-8") as f:
    data_samples = json.load(f)

print("Before:", type(data_samples), "len(dict) =", len(data_samples))

# Convert dict → list of values
data_samples = list(data_samples.values())

print("After:", type(data_samples), "len(list) =", len(data_samples))
print("Example sample:", data_samples[0])


Before: <class 'dict'> len(dict) = 18
After: <class 'list'> len(list) = 18
Example sample: {'sentences': ['You will be asked a multiple-choice question. Respond with the letter which corresponds to the correct answer,followed by a period. There is no need to provide an explanation, so your response should be very short.\nNow here is the question:\nThe energy given up by electrons as they move through the electron transport chain is used to\n A. make glucose\n B. break down glucose\n C. produce ATP\n D. make NADH\n\nResponse:', 'Below is a multiple-choice question. Respond with the letter that best answers the question. Keep your response brief, stating only the letter corresponding to your answer, followed by a period, with no explanation.\nThe question is:\nThe energy given up by electrons as they move through the electron transport chain is used to\n A. make NADH\n B. break down glucose\n C. produce ATP\n D. make glucose\n\nResponse:', 'Prepare to answer a multiple-choice question. P

In [19]:
data_samples = data_samples[:3]  # or 5, 10 etc.
print("Using", len(data_samples), "samples")


Using 3 samples


In [20]:
!grep -n "model.to" keyneuron/neuron_attribution/_neuron_attribution.py


47:        # ✅ explicit tensor device (NOT model.to)
432:                # then patch the model to replace the activations with the scaled activations
442:                # then forward through the model to get the logits


In [21]:
!grep -R "model.to(" -n .
!grep -R ".to(self.model" -n .
!grep -R ".to(self.model.device" -n .



./Untitled.ipynb:128:      "./Untitled.ipynb:139:    \"!grep -R \\\"model.to(\\\" -n .\\n\",\n",
./Untitled.ipynb:152:    "!grep -R \"model.to(\" -n .\n",
./Untitled.ipynb:129:      "./Untitled.ipynb:127:      \"./keyneuron/neuron_attribution/_neuron_attribution.py:93:            encoded_input = self.tokenizer(prompt, return_tensors=\\\"pt\\\").to(self.model.device)\\n\",\n",
./Untitled.ipynb:130:      "./Untitled.ipynb:128:      \"./keyneuron/neuron_attribution/_neuron_attribution.py:224:        encoded_input = self.tokenizer(prompt, return_tensors=\\\"pt\\\").to(self.model.device)\\n\",\n",
./Untitled.ipynb:131:      "./Untitled.ipynb:129:      \"./keyneuron/neuron_attribution/.ipynb_checkpoints/_neuron_attribution-checkpoint.py:93:            encoded_input = self.tokenizer(prompt, return_tensors=\\\"pt\\\").to(self.model.device)\\n\",\n",
./Untitled.ipynb:132:      "./Untitled.ipynb:130:      \"./keyneuron/neuron_attribution/.ipynb_checkpoints/_neuron_attribution-checkpoint.py:224: 

In [22]:
!rm -rf keyneuron/neuron_attribution/.ipynb_checkpoints


In [ ]:
import sys
!{sys.executable} -m pip install bitsandbytes
